#Загрузка данных

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import log_loss, roc_auc_score

In [6]:
data = pd.read_csv('/content/drive/MyDrive/recsys/data.csv')
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


In [7]:
# убираем дубликаты из датасета
data.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [8]:
#убираем неиспользуемые колонки
data.drop(columns=['banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'], inplace=True)
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,1,1


In [9]:
#убираем колонку impressions
data.drop(columns=['impressions'], inplace=True)

In [10]:
#добавляем колонку дата показа
data['date'] = pd.to_datetime(data['date_time']).dt.date

In [11]:
#добавляем колонку время показа
data['hour'] = pd.to_datetime(data['date_time']).dt.hour

In [12]:
#убираем колонку date_time
data.drop(columns = ['date_time'], inplace=True)

In [13]:
from datetime import date
#разделим данные на обучающую, тестовую и валидационную выборки
#валидационная нужна для подбора гиперпараметров
test_date = data['date'].max() #последний день
val_date = date(2021, 10, 1) #предпоследний день в датасете

train = data[data['date'] < val_date]
test = data[data['date'] == test_date]
val = data[data['date'] == val_date]

print(len(train))
print(len(test))
print(len(val))

12041489
2127795
1642557


In [14]:
#уберем ненужную колонку date
train = train.drop(columns=['date'])
test = test.drop(columns=['date'])
val = val.drop(columns=['date'])

#Baseline

In [ ]:
mean_value = y_train.mean()
y_pred = [mean_value] * len(y_test)
logloss = log_loss(y_test, y_pred)
print(f"baseline logloss: {logloss}")
auc = roc_auc_score(y_test, y_pred)
print(f"baseline auc: {auc}")

baseline logloss: 0.15407879603434949
baseline auc: 0.5


#Логистическая регрессия из HW1

В качестве модели возьмем логистическую регрессию с l2-регуляризацией

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    solver = 'liblinear'
)

In [ ]:
model = model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict_proba(X_test)[:, 1]

In [ ]:
logloss = log_loss(y_test, y_pred)
print(f"logloss: {logloss}")
auc = roc_auc_score(y_test, y_pred)
print(f"auc: {auc}")

logloss: 0.1325096903606202
auc: 0.7808475982975257


получилось побить бейзлайн

#FFM

In [12]:
#устанавливаем xlearn, откуда возьмем имплементацию FFM
!pip install xlearn

Сначала приведем датасет в libffm формат. Код взят отсюда https://github.com/wngaw/blog/blob/master/xlearn_example/src/utils.py

In [11]:
import json
import math


def _convert_to_ffm(path, df, type, target, numerics, categories, features, encoder):
    # Flagging categorical and numerical fields
    print('convert_to_ffm - START')
    for x in numerics:
        if(x not in encoder['catdict']):
            #print(f'UPDATING CATDICT: numeric field - {x}')
            encoder['catdict'][x] = 0
    for x in categories:
        if(x not in encoder['catdict']):
            #print(f'UPDATING CATDICT: categorical field - {x}')
            encoder['catdict'][x] = 1

    nrows = df.shape[0]
    with open(path + str(type) + "_ffm.txt", "w") as text_file:

        # Looping over rows to convert each row to libffm format
        for n, r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow[target]))  # Set Target Variable here

            # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(encoder['catdict'].keys()):
                if(encoder['catdict'][x] == 0):
                    # Not adding numerical values that are nan
                    if math.isnan(datarow[x]) is not True:
                        datastring = datastring + " "+str(i)+":" + str(i)+":" + str(datarow[x])
                else:

                    # For a new field appearing in a training example
                    if(x not in encoder['catcodes']):
                        #print(f'UPDATING CATCODES: categorical field - {x}')
                        encoder['catcodes'][x] = {}
                        encoder['currentcode'] += 1
                        #print(f'UPDATING CATCODES: categorical value for field {x} - {datarow[x]}')
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    # For already encoded fields
                    elif(datarow[x] not in encoder['catcodes'][x]):
                        encoder['currentcode'] += 1
                        #print(f'UPDATING CATCODES: categorical value for field {x} - {datarow[x]}')
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    code = encoder['catcodes'][x][datarow[x]]
                    datastring = datastring + " "+str(i)+":" + str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

    # print('Encoder Summary:')
    # print(json.dumps(encoder, indent=4))
    return encoder

In [14]:
NUMERICAL_FEATURES = ['campaign_clicks', 'hour']
CATEGORICAL_FEATURES = ['os_id', 'country_id', 'zone_id', 'banner_id', 'oaid_hash']
ALL_FEATURES =  ['os_id', 'country_id', 'zone_id', 'banner_id', 'oaid_hash', 'campaign_clicks', 'hour']
GOAL = ['clicks']


encoder = {"currentcode": len(NUMERICAL_FEATURES),
           "catdict": {},
           "catcodes": {}}

In [15]:
encoder = _convert_to_ffm('', train, 'train', GOAL[0],
                          NUMERICAL_FEATURES,
                          CATEGORICAL_FEATURES,
                          ALL_FEATURES,
                          encoder)

convert_to_ffm - START


In [16]:
encoder = _convert_to_ffm('', test, 'test', GOAL[0],
                          NUMERICAL_FEATURES,
                          CATEGORICAL_FEATURES,
                          ALL_FEATURES,
                          encoder)

convert_to_ffm - START


In [17]:
encoder = _convert_to_ffm('', val, 'val', GOAL[0],
                          NUMERICAL_FEATURES,
                          CATEGORICAL_FEATURES,
                          ALL_FEATURES,
                          encoder)

convert_to_ffm - START


In [4]:
import xlearn as xl

ffm_model = xl.create_ffm()
ffm_model.setTrain("/content/train_ffm.txt")
param = {'task': 'binary',
         'lr': 0.2,
         'lambda': 0.002,
         'metric': 'auc'}

In [1]:
#без этого xlearn не работает в колабе
import os
os.environ['USER'] = 'test'

In [5]:
ffm_model.fit(param, './model.out')

In [24]:
ffm_model.setTest("/content/test_ffm.txt")
ffm_model.setSigmoid() # Convert output to 0-1

# Start to predict
# The output result will be stored in output.txt
ffm_model.predict("./model.out", "./output.txt")

In [25]:
y_pred = pd.read_csv("output.txt", header=None)
y_test = test['clicks']

logloss = log_loss(y_test, y_pred)
print(f"logloss: {logloss}")
auc = roc_auc_score(y_test, y_pred)
print(f"auc: {auc}")

logloss: 0.1492597106388284
auc: 0.7528505949778825


видим, что логлосс и auc немного хуже, чем у линейной модели из первой домашки. попробуем улучшить метрики, подобрав гиперпараметры (коэффициент регуляризации lambda и размерность латентного пространства k) модели.

In [15]:
#Сначала подберем оптимальный параметр k
import xlearn as xl

k_grid = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for k in k_grid:
  print(f"k = {k}")
  ffm_model = xl.create_ffm()
  ffm_model.setTrain("/content/train_ffm.txt")
  param = {'task': 'binary',
            'k': k,
            'metric': 'auc'}
  MODEL_NAME = f"model_{k}.out"
  ffm_model.fit(param, f'./{MODEL_NAME}')

  ffm_model.setTest("/content/val_ffm.txt")
  ffm_model.setSigmoid()
  ffm_model.predict(f"./{MODEL_NAME}", "./output.txt")

  y_pred = pd.read_csv("output.txt", header=None)
  y_test = val['clicks']

  logloss = log_loss(y_test, y_pred)
  print(f"logloss: {logloss}")
  auc = roc_auc_score(y_test, y_pred)
  print(f"auc: {auc}")

k = 1
logloss: 0.16218607549673195
auc: 0.780922604261618
k = 2
logloss: 0.1615562583910295
auc: 0.7819587507502028
k = 3
logloss: 0.15994091779299963
auc: 0.7847911341773649
k = 4
logloss: 0.15949835311836089
auc: 0.7856908582660176
k = 5
logloss: 0.1593282227211086
auc: 0.7864133515854739
k = 6
logloss: 0.15923544263503336
auc: 0.7861907380548316
k = 7
logloss: 0.15921015323451687
auc: 0.7864210782767111
k = 8
logloss: 0.1590032212032277
auc: 0.7869808834531536
k = 9
logloss: 0.15946850271546442
auc: 0.7871642544004769
k = 10
logloss: 0.1585866997203471
auc: 0.7878037158097543


Видим, что лучшее значение метрик достигается при k = 10. Скорее всего, можно получить еще лучшее значение метрик при k > 10, но в FFM мы выучиваем эмбеддинги для всех взаимодействий фичей друг с другом (то есть для n фичей суммарно n^2 эмбеддингов), поэтому размерность латентного пространства должна быть небольшой, поэтому остановимся на k = 10.

In [13]:
#теперь подберем лямбду, ее дефолтное значение в xlearn = 0.00002
import xlearn as xl

for l in [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]:
  print(f"lambda = {l}")
  ffm_model = xl.create_ffm()
  ffm_model.setTrain("/content/train_ffm.txt")
  param = {'task': 'binary',
            'lambda': l,
           'k': 10,
            'metric': 'auc'}
  MODEL_NAME = f"model_{l}.out"
  ffm_model.fit(param, f'./{MODEL_NAME}')

  ffm_model.setTest("/content/val_ffm.txt")
  ffm_model.setSigmoid()
  ffm_model.predict(f"./{MODEL_NAME}", "./output.txt")

  y_pred = pd.read_csv("output.txt", header=None)
  y_test = val['clicks']

  logloss = log_loss(y_test, y_pred)
  print(f"logloss: {logloss}")
  auc = roc_auc_score(y_test, y_pred)
  print(f"auc: {auc}")

lambda = 1e-05
logloss: 0.16376672093609465
auc: 0.7859209130678468
lambda = 0.0001
logloss: 0.15369141388657498
auc: 0.7926610476381074
lambda = 0.001
logloss: 0.17032483158762915
auc: 0.7461716371363983
lambda = 0.01
logloss: 0.1764815824883843
auc: 0.7379001352291226
lambda = 0.1
logloss: 0.1786247740544869
auc: 0.7458013272954983
lambda = 1
logloss: 0.17929805061517146
auc: 0.7199720912025233


Видим, что лучшее значение auc достигается при lambda = 0.0001. Протестируем финальную модель с k = 10 и lambda = 0.0001

In [3]:
import xlearn as xl

ffm_model = xl.create_ffm()
ffm_model.setTrain("/content/train_ffm.txt")
param = {'task': 'binary',
          'lambda': 0.0001,
          'k': 10,
          'metric': 'auc'}

ffm_model.setTest("/content/test_ffm.txt")
ffm_model.setSigmoid() # Convert output to 0-1
ffm_model.fit(param, f'./model.out')

In [15]:
ffm_model.predict("./model.out", "./output.txt")
y_pred = pd.read_csv("output.txt", header=None)
y_test = test['clicks']

logloss = log_loss(y_test, y_pred)
print(f"logloss: {logloss}")
auc = roc_auc_score(y_test, y_pred)
print(f"auc: {auc}")

logloss: 0.13556217070556895
auc: 0.7781032937739147


Получили значения logloss и auc чуть хуже, чем у логистической регрессии, но очень близкие к ним